In [1]:
import os
os.environ['HF_HOME'] = '/scratch/singh/hf/'

### Task 1

#### Loading the dataset and making another split for testing

In [2]:
from datasets import load_dataset
import torch

# Load the dataset (assuming the dataset path is 'batterydata/pos_tagging')
dataset = load_dataset("batterydata/pos_tagging")

dataset_split = dataset["train"].train_test_split(test_size=0.1, shuffle=True)
dataset["validation"] = dataset_split["test"]
dataset["train"] = dataset_split["train"]
print(dataset)


README.md:   0%|          | 0.00/587 [00:00<?, ?B/s]

e:\Uni Stuttgart\课程\Intro to DL\my_env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\scratch\singh\hf\hub\datasets--batterydata--pos_tagging. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train.json:   0%|          | 0.00/5.05M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/601k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13054 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1451 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['words', 'labels'],
        num_rows: 11748
    })
    test: Dataset({
        features: ['words', 'labels'],
        num_rows: 1451
    })
    validation: Dataset({
        features: ['words', 'labels'],
        num_rows: 1306
    })
})


#### Same indexing as before for class to index and index to class

In [3]:
labels_unique = list(
    set([label for sample in dataset["train"] for label in sample["labels"]])
)
print(labels_unique)
print(f"Number of classes: {len(labels_unique)}")
ctoi = {label: idx for idx, label in enumerate(labels_unique)}
itoc = {idx: label for label, idx in ctoi.items()}
print(ctoi)
print(itoc)

['RBS', 'CC', 'CD', 'PRP', 'VB', 'LS', 'JJ', '-LRB-', 'NNS', '``', "''", 'JJR', 'JJS', '(', ')', 'WP', 'IN', 'UH', 'SYM', 'POS', 'NNP', '-RRB-', 'VBN', '$', ',', 'WDT', 'PDT', 'WRB', 'TO', 'RP', 'DT', '-NONE-', ':', 'RB', 'PRP$', 'VBG', 'RBR', 'MD', 'NN', 'EX', 'VBD', 'FW', 'VBP', 'NNPS', 'WP$', '#', 'VBZ', '.']
Number of classes: 48
{'RBS': 0, 'CC': 1, 'CD': 2, 'PRP': 3, 'VB': 4, 'LS': 5, 'JJ': 6, '-LRB-': 7, 'NNS': 8, '``': 9, "''": 10, 'JJR': 11, 'JJS': 12, '(': 13, ')': 14, 'WP': 15, 'IN': 16, 'UH': 17, 'SYM': 18, 'POS': 19, 'NNP': 20, '-RRB-': 21, 'VBN': 22, '$': 23, ',': 24, 'WDT': 25, 'PDT': 26, 'WRB': 27, 'TO': 28, 'RP': 29, 'DT': 30, '-NONE-': 31, ':': 32, 'RB': 33, 'PRP$': 34, 'VBG': 35, 'RBR': 36, 'MD': 37, 'NN': 38, 'EX': 39, 'VBD': 40, 'FW': 41, 'VBP': 42, 'NNPS': 43, 'WP$': 44, '#': 45, 'VBZ': 46, '.': 47}
{0: 'RBS', 1: 'CC', 2: 'CD', 3: 'PRP', 4: 'VB', 5: 'LS', 6: 'JJ', 7: '-LRB-', 8: 'NNS', 9: '``', 10: "''", 11: 'JJR', 12: 'JJS', 13: '(', 14: ')', 15: 'WP', 16: 'IN', 1

#### Initializing model and tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Use DistilRoBERTa (a smaller version of RoBERTa)
model_name = "distilbert/distilroberta-base"

# Load tokenizer and model
# add_prefix_space=True 参数确保在分词时在单词前添加空格
# num_labels: 指定了分类标签的数量，使用 len(labels_unique) 计算
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(labels_unique), id2label=itoc, label2id=ctoi)
# 这段代码的目的是为token分类任务（如命名实体识别）准备模型和tokenizer。
# 通过使用预训练的DistilRoBERTa模型，你可以在此基础上进行微调，以适应特定的分类任务。

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

e:\Uni Stuttgart\课程\Intro to DL\my_env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\scratch\singh\hf\hub\models--distilbert--distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Now we need to align the tokenization output of the tokenizer to the labels as they do sub word tokenization
#### Then finally, we will map this function to the datasets

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["words"],  # Words are already tokenized
        truncation=True, # 允许截断过长的序列
        padding="max_length", # 填充到最大长度
        is_split_into_words=True # 表明输入已经按词分割
    )

    all_labels = []
    for batch_idx, word_labels in enumerate(examples["labels"]):  # Iterate over batch
        # word_ids() 是 tokenized_inputs 对象的一个方法。
        # 这个方法返回一个列表，表示每个 token 对应的原始单词的索引
        word_ids = tokenized_inputs.word_ids(batch_index=batch_idx)  # Get word IDs
        # previous_word_idx 用于跟踪上一个处理的词的索引。这在处理子词（subwords）时特别重要
        previous_word_idx = None
        labels = []

        for word_idx in word_ids:
            if word_idx is None:
                labels.append(-100)  # Ignore special tokens
                # 当 word_idx != previous_word_idx 时，表示遇到了一个新词的开始，此时我们为这个新词分配标签
            elif word_idx != previous_word_idx:
                labels.append(ctoi[word_labels[word_idx]])  # Assign label to first subword
            # 如果 word_idx == previous_word_idx（且不为 None），说明当前 token 是前一个词的子词，
            # 我们通常会忽略它（在 NER 任务中给予特殊标记 -100）。
            else:
                labels.append(-100)  # Ignore subword tokens
            previous_word_idx = word_idx
        '''重要性：

在处理使用子词 tokenization 的模型（如 BERT、RoBERTa 等）时，这种方法非常重要。
它确保了在词被分割成多个 token 的情况下，只有第一个 token 获得实际的标签，而其他的子词 token 被适当地忽略。
这种处理方式有助于保持原始标注的完整性，同时适应子词 tokenization 的特性。'''
        all_labels.append(labels)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs
'''tokenize_and_align_labels 函数返回的是一个经过处理的字典（dictionary），
包含了tokenized的输入和对齐后的标签。
让我详细解释一下这个函数的返回值。其中，labels 字段是函数处理的关键部分。
它包含了与 input_ids 长度相同的标签列表。
对于每个token，如果它是一个词的第一个子词，会被赋予相应的标签。
对于子词（非词首token）和特殊token（如[CLS], [SEP]），标签被设置为 -100。'''
# Apply tokenization with batched processing
# 对于训练集和测试集中的每个样本（或每批样本），map 方法会调用 tokenize_and_align_labels 函数。
# 这个函数会对输入文本进行分词，并将原始的词级别标签对齐到 token 级别。
# 处理后的数据将包含分词后的输入和对应的 token 级别标签。
train_dataset = dataset["train"].map(tokenize_and_align_labels, batched=True)
test_dataset = dataset["test"].map(tokenize_and_align_labels, batched=True)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["words"])
test_dataset = test_dataset.remove_columns(["words"])

# Convert to PyTorch tensors
# 这是Hugging Face datasets库中Dataset对象的一个方法。
# 它用于指定数据集应该以何种格式返回数据。
# "torch" 参数：指定要将数据转换为PyTorch张量格式。
train_dataset.set_format("torch")
test_dataset.set_format("torch")


Map:   0%|          | 0/11748 [00:00<?, ? examples/s]

Map:   0%|          | 0/1451 [00:00<?, ? examples/s]

In [6]:
train_dataset.column_names

['labels', 'input_ids', 'attention_mask']

In [7]:
print("CUDA Available:", torch.cuda.is_available())
print("Using device:", model.device)

CUDA Available: True
Using device: cpu


#### Setting up test dataloader for evaluation

In [8]:
from torch.utils.data import DataLoader
import torch

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Move model to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              

#### Defining a function for computing accuracy
#### It'll evaluate the model on the test dataset/given dataloader

In [ ]:
def compute_accuracy(model, dataloader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            # outputs.logits:

            # 这是模型输出的一部分，通常是一个三维张量。
            # 形状通常为 (batch_size, sequence_length, num_classes)。
            # 每个元素代表模型对每个token属于每个类别的预测分数（或logits）。
            # logits 是模型神经网络最后一层（通常是全连接层）的原始输出。
            # 它们是未经过softmax或其他激活函数处理的原始scores。
            predictions = torch.argmax(outputs.logits, dim=-1)  # Get most likely labels

        # Flatten both predictions and labels
        # 这行创建了一个布尔掩码，用于识别有效的（非填充）标签。
        # 在之前的处理中，-100 被用作填充标签和特殊 token 的标签。
        # 这个掩码将在真实标签不是 -100 的位置为 True，否则为 False。
        mask = labels != -100  # Ignore padded labels
        # predictions[mask] 操作：
        # 这个操作会创建一个新的一维张量。
        # 新张量只包含 predictions 中对应 mask 为 True 的元素。
        '''predictions = [[1, 2, 3, 0],
               [2, 1, 0, 0]]
mask = [[True, True, True, False],
        [True, True, False, False]]
        那么 predictions[mask] 的结果将是[1, 2, 3, 2, 1]'''
        correct += (predictions[mask] == labels[mask]).sum().item()
        total += mask.sum().item()

    accuracy = correct / total if total > 0 else 0
    return accuracy

# Run evaluation using the DataLoader
accuracy = compute_accuracy(model, test_dataloader)
print(f"POS Tagging Accuracy on Test Set: {accuracy:.4f}")


POS Tagging Accuracy on Test Set: 0.0110


#### Finally, defining the training arguments for the trainer and initialzing the trainer

In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./pos_tagging_model", # 模型和检查点的保存目录
    evaluation_strategy="epoch", # 每个 epoch 结束后进行评估
    save_strategy="epoch",  # 每个 epoch 结束后保存模型
    per_device_train_batch_size=8,  # 每个 epoch 结束后保存模型
    per_device_eval_batch_size=8, # 每个设备的评估批次大小
    num_train_epochs=3, # 训练的总 epoch 数
    weight_decay=0.01, # L2 正则化系数
    save_total_limit=2, # 最多保存的检查点数量
    report_to="none", # 不使用任何报告工具（如 TensorBoard）
    logging_strategy="epoch" # 每个 epoch 结束后记录日志
)

trainer = Trainer(
    model=model, # 要训练的模型
    args=training_args, # 上面定义的训练参数
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

e:\Uni Stuttgart\课程\Intro to DL\my_env\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.180400,0.087213
2,0.051800,0.092754
3,0.031600,0.093473


TrainOutput(global_step=4407, training_loss=0.08793052654296653, metrics={'train_runtime': 8099.8094, 'train_samples_per_second': 4.351, 'train_steps_per_second': 0.544, 'total_flos': 4608567785226240.0, 'train_loss': 0.08793052654296653, 'epoch': 3.0})

In [11]:
accuracy = compute_accuracy(model, test_dataloader)
print(f"POS Tagging Accuracy on Test Set: {accuracy:.4f}")

POS Tagging Accuracy on Test Set: 0.9762


In [12]:
# Evaluate on test set
# evaluate() 方法在评估数据集（通常是测试集）上运行模型，并计算各种性能指标。
# 它返回一个字典，其中包含评估指标的名称和对应的值。
results = trainer.evaluate()
results

{'eval_loss': 0.09347294270992279,
 'eval_runtime': 81.5145,
 'eval_samples_per_second': 17.801,
 'eval_steps_per_second': 2.233,
 'epoch': 3.0}

### Task 2

In [13]:
from transformers import pipeline

In [14]:
# Load the pipeline
# 这是Hugging Face Transformers库中的一个高级API，用于简化模型的使用。
# 它自动处理模型加载、tokenization和推理过程。
# 指定了pipeline的任务类型为文本生成。
# 这意味着模型将被用于生成连续的文本。
pipe = pipeline(
    "text-generation",
    # 指定了pipeline的任务类型为文本生成。
    # 这意味着模型将被用于生成连续的文本。
    model="microsoft/Phi-3.5-mini-instruct",
    trust_remote_code=True,  # Trust the remote code; this is required for some models, but always check the code first!
    device="cpu",  # Set this to "cuda" for GPU acceleration if available
    torch_dtype=torch.bfloat16,  # Use bfloat16 for less memory usage and faster inference
)

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

e:\Uni Stuttgart\课程\Intro to DL\my_env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\scratch\singh\hf\hub\models--microsoft--Phi-3.5-mini-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Could not load model microsoft/Phi-3.5-mini-instruct with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.phi3.modeling_phi3.Phi3ForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "e:\Uni Stuttgart\课程\Intro to DL\my_env\Lib\site-packages\transformers\pipelines\base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Uni Stuttgart\课程\Intro to DL\my_env\Lib\site-packages\transformers\models\auto\auto_factory.py", line 559, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Uni Stuttgart\课程\Intro to DL\my_env\Lib\site-packages\transformers\modeling_utils.py", line 4245, in from_pretrained
    ) = cls._load_pretrained_model(
      

In [15]:
def generate_text(prompt, max_new_tokens=50):
    # Tokenize input
    # 指定返回的结果应该是PyTorch张量。
    # "pt"代表PyTorch。其他选项包括"tf"（TensorFlow）或"np"（NumPy数组）。
    inputs = tokenizer(prompt, return_tensors="pt")

    # Move ALL input tensors to the same device as the model
    for key in inputs.keys():
        inputs[key] = inputs[key].to(device)

    # Manually create position IDs and move them to the correct device
    # 获取输入序列的长度。
    # inputs["input_ids"] 是一个形状为 (batch_size, sequence_length) 的张量。
    # .shape[1] 取第二个维度，即序列长度。
    # torch.arange(...):
    # 创建一个从 0 到 (序列长度 - 1) 的整数序列。
    # 例如，如果序列长度是 5，这将创建 [0, 1, 2, 3, 4]。
    # 指定生成的张量数据类型为长整型
    position_ids = torch.arange(inputs["input_ids"].shape[1], dtype=torch.long).unsqueeze(0).to(device)

    # Generate output while ensuring ALL tensors are on the correct device
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens
        )
    # tokenizer.decode():
    # 这是tokenizer的一个方法，用于将数字token ID序列转换回原始文本。
    # 它执行的是tokenization的逆操作。
    # 这个参数指示解码过程中要跳过特殊token。
    # 特殊token包括如 [PAD], [CLS], [SEP], [MASK] 等，这些通常不应该出现在最终的文本输出中。
    # 设置为 True 可以得到更干净、更可读的输出。
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
'''处理过程：

将数字ID序列映射回对应的token（单词或子词）。
将这些token拼接成一个连续的字符串。
移除所有的特殊token（如果 skip_special_tokens=True）。
处理任何需要的后处理步骤（如合并分割的单词）。
返回值：

一个字符串，包含解码后的文本。
用途：

在文本生成任务中，这通常是获取最终可读输出的最后一步。
它将模型的数值输出转换为人类可理解的文本。
示例： 假设 outputs[0] 是 [7993, 2000, 10169, 58, 7592, 11, 2129, 2024, 2017, 30] 
解码后可能得到类似 "Translate to French: Hello, how are you?" 的文本。'''


'处理过程：\n\n将数字ID序列映射回对应的token（单词或子词）。\n将这些token拼接成一个连续的字符串。\n移除所有的特殊token（如果 skip_special_tokens=True）。\n处理任何需要的后处理步骤（如合并分割的单词）。\n返回值：\n\n一个字符串，包含解码后的文本。\n用途：\n\n在文本生成任务中，这通常是获取最终可读输出的最后一步。\n它将模型的数值输出转换为人类可理解的文本。\n示例： 假设 outputs[0] 是 [7993, 2000, 10169, 58, 7592, 11, 2129, 2024, 2017, 30] \n解码后可能得到类似 "Translate to French: Hello, how are you?" 的文本。'

In [16]:
# Example prompts to test model behavior
prompts = [
    "Write a short story about a robot discovering emotions.",
    "Explain how transformers work in deep learning.",
    "You are a helpful assistant. Answer: What is deep learning?"
]

# Generate text for each prompt
for prompt in prompts:
    output = pipe(prompt, max_new_tokens=100)
    print(f"\nPrompt: {prompt}")
    print(f"Generated Output: {output}\n")


NameError: name 'pipe' is not defined

In [ ]:
output = pipe(
    "Explain quantum physics in simple terms.",
    max_new_tokens=100,  # Control text length
    temperature=0.7,  # Higher values = more randomness
    top_p=0.9,  # Use nucleus sampling for diversity
    do_sample=True  # Enable sampling instead of greedy decoding
)
'''top_p=0.9:
这是核采样（nucleus sampling）的参数，也称为 top-p 采样。
它只考虑累积概率达到 top_p 的最可能的 token。
0.9 意味着模型将从累积概率达到 90% 的最可能 token 中进行采样。
这有助于保持多样性，同时避免选择非常不可能的 token。

do_sample=True:
启用采样而不是贪婪解码。
当设置为 True 时，模型会根据概率分布进行采样，而不是总是选择最可能的下一个 token。
这允许生成更多样化的文本，而不是每次都产生相同的输出。
'''
print("\nGenerated Output with Custom Parameters:")
print(output[0]['generated_text'])



Generated Output with Custom Parameters:
Explain quantum physics in simple terms.

## Answer:
Quantum physics is a branch of science that studies the behavior of the smallest particles in the universe, like atoms and photons (which are particles of light). Unlike the large objects we see in our daily life, these tiny particles don't follow the usual rules we expect. Here are some key points about quantum physics:

1. **Quantum Superposition**: This is the idea that particles can exist in multiple states at once. For example,


In [ ]:
zero_shot_prompt = """Translate the following sentences from English to French:

1. I love programming.
2. The weather is beautiful. 
3. I love deep learning. """
'''output 是pipeline返回的结果，通常是一个列表。
output[0] 取第一个（也可能是唯一的）生成结果。
['generated_text'] 访问生成的文本内容。'''
output = pipe(zero_shot_prompt, max_new_tokens=100)
print("\nFew-Shot Learning Output:")
print(output[0]['generated_text'])
'''预期输出：

输出将包括原始的提示文本（英语句子）以及模型生成的法语翻译。
由于使用了zero-shot方法，模型是在没有特定翻译示例的情况下尝试完成任务的。'''


Few-Shot Learning Output:
Translate the following sentences from English to French:

1. I love programming.
2. The weather is beautiful. 
3. I love deep learning. 
4. The sky is clear tonight.
5. I am learning French.

# Answer

1. J'adore programmer.
2. Le temps est magnifique.
3. J'adore l'apprentissage profond.
4. Le ciel est dégagé ce soir.
5. J'apprends le français.


In [22]:
few_shot_prompt = """Translate the third sentence from English to French, 
the first two are examples on how to translate them:

1. I love programming. - J’aime la programmation.
2. The weather is beautiful. - Il fait beau.
3. I love deep learning. -"""

output = pipe(few_shot_prompt, max_new_tokens=40)
print("\nFew-Shot Learning Output:")
print(output[0]['generated_text'])



Few-Shot Learning Output:
Translate the third sentence from English to French, 
the first two are examples on how to translate them:

1. I love programming. - J’aime la programmation.
2. The weather is beautiful. - Il fait beau.
3. I love deep learning. - 

# Answer
Je t'aime l'apprentissage profond.

Note: In French, possessive structures can be a bit different from English. Instead of


In [20]:
zero_shot_prompt = """Summarize the following article in one sentence: 
An ANN consists of connected units or nodes called artificial neurons, 
which loosely model the neurons in the brain. Artificial neuron models that mimic 
biological neurons more closely have also been recently investigated and shown 
to significantly improve performance. These are connected by edges, which model the synapses in the brain. 
Each artificial neuron receives signals from connected neurons, then processes them and 
sends a signal to other connected neurons. The "signal" is a real number, 
and the output of each neuron is computed by some non-linear function of the sum of its inputs, 
called the activation function. The strength of the signal at each connection is 
determined by a weight, which adjusts during the learning process. 
Typically, neurons are aggregated into layers. Different layers may perform 
different transformations on their inputs. Signals travel from the first layer (the input layer) 
to the last layer (the output layer), possibly passing through multiple intermediate layers (hidden layers). 
A network is typically called a deep neural network if it has at least two hidden layers. 
Artificial neural networks are used for various tasks, including predictive modeling, 
adaptive control, and solving problems in artificial intelligence. 
They can learn from experience, and can derive conclusions from a complex and 
seemingly unrelated set of information."""
output = pipe(zero_shot_prompt, max_new_tokens=100)

print("\nZero-Shot Learning Output:")
print(output[0]['generated_text'])



Zero-Shot Learning Output:
Summarize the following article in one sentence: 
An ANN consists of connected units or nodes called artificial neurons, 
which loosely model the neurons in the brain. Artificial neuron models that mimic 
biological neurons more closely have also been recently investigated and shown 
to significantly improve performance. These are connected by edges, which model the synapses in the brain. 
Each artificial neuron receives signals from connected neurons, then processes them and 
sends a signal to other connected neurons. The "signal" is a real number, 
and the output of each neuron is computed by some non-linear function of the sum of its inputs, 
called the activation function. The strength of the signal at each connection is 
determined by a weight, which adjusts during the learning process. 
Typically, neurons are aggregated into layers. Different layers may perform 
different transformations on their inputs. Signals travel from the first layer (the input l